# Initial Data Preparation for CT413 FYP

1. Mounting Google Drive
2. Read in individual data files
3.

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


### Mounting Google Drive

This will popup a prompt for permission to access google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


### Read in individual data files

In [ ]:
import pandas as pd
import dask.dataframe as dd

# Path to folder of files
folder_path = '/content/drive/MyDrive/SleepHealthDataTables/'

# TODO:
#
# Read in certain columns from the data
df_aeb = dd.read_csv(folder_path + 'active-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
df_aet = dd.read_csv(folder_path + 'apple-exercise-time.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
df_beb = dd.read_csv(folder_path + 'basal-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
df_bm = dd.read_csv(folder_path + 'body-mass.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
df_dwr = dd.read_csv(folder_path + 'distance-walking-running.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
df_fcl = dd.read_csv(folder_path + 'flights-climbed.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
df_hr = dd.read_csv(folder_path + 'heart-rate.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
'''
df_h = dd.read_csv(folder_path + 'height.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
df_nt = dd.read_csv(folder_path + 'nap-tracker.csv', usecols=['participantId', 'timestamp', 'NapDuration', 'NapQuality'], assume_missing=True)
df_sqc = dd.read_csv(folder_path + 'sleep-quality-checker.csv', usecols=['participantId', 'timestamp', 'sq_score'], assume_missing=True)
df_sc = dd.read_csv(folder_path + 'step-count.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], assume_missing=True)
'''

timestamped_dfs = [df_aeb, df_aet, df_beb, df_bm, df_dwr, df_fcl, df_hr]
#, df_h, df_nt, df_sqc, df_sc]

# Make a list of Strings with the names of the dataframes
df_names = ['active_energy_burned', 'apple_exercise_time', 'basal_energy_burned', 'body_mass', 'distance_walking_running', 'flights_climbed', 'heart_rate', 'height', 'nap_tracker', 'sleep_quality_checker', 'step_count']



In [ ]:
# Sort each dataframe by participantId
for i, df in enumerate(timestamped_dfs):
    timestamped_dfs[i] = df.sort_values('participantId')
    # Convert startTime and endTime to datetime if they are in the df
    if 'startTime' and 'endTime' in df:
      df['timestamp'] = dd.to_datetime(df['timestamp'])
    df['timestamp'] = dd.to_datetime(df['timestamp'])


In [ ]:
import os

# Merge and print the dataframe
#
# TODO:
# When two tables are merged, delete df and original table from drive, and save new merged table
#
merged_timestamped = timestamped_dfs[0]
for df in timestamped_dfs[1:]:
  merged_timestamped = dd.merge(merged_timestamped, df, on=['participantId', 'timestamp'], how='outer', suffixes=('', '_y'))
  del df

merged_timestamped = merged_timestamped.compute()

print("Merged Timestamped DataFrame:")
print(merged_timestamped)


In [ ]:
# Create summary statistics for each dataframe for value column
for idx, curr_df in enumerate(timestamped_dfs, start=1):
    # Print the name of the dataframe
    print(f"\nDataFrame {idx} Name: {df_names[idx-1]}")
    if 'value' in curr_df.columns:
        print(curr_df['value'].describe())
    elif 'sq_score' in curr_df.columns:
        print(curr_df['sq_score'].describe())
    elif 'NapQuality' in curr_df.columns:
        print(curr_df['NapQuality'].describe())
        print(curr_df['NapDuration'].describe())